#Przygotowanie środowisk

In [2]:
#!pip install autogluon

In [3]:
import numpy as np
import pandas as pd

In [4]:
from autogluon.tabular import TabularPredictor

d:\Data\Dokumenty\kodzenie\Jula\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from sklearn.model_selection import train_test_split

#Dane

In [6]:
url_train = "https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.data"
url_test = "https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_test.data"
url_train_labels = "https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.labels"

In [7]:
data_train = pd.read_csv(url_train, header=None, sep=' ')
data_test = pd.read_csv(url_test, header=None, sep=' ')
labels_train = pd.read_csv(url_train_labels, header=None)

In [8]:
data_train = data_train.drop(columns=[500], axis=1)
data_test = data_test.drop(columns=[500], axis=1)

In [9]:
# Split the data into training and validation sets
data_train, data_val, labels_train, labels_val = train_test_split(data_train, labels_train, test_size=0.2, random_state=42)

In [10]:
labels_train = labels_train.rename(columns={0: 'Y'})

In [11]:
labels_val = labels_val.rename(columns={0: 'Y'})

In [12]:
labels_train

,Y
968,1
240,1
819,1
692,1
420,-1
...,...
1130,1
1294,1
860,1
1459,-1


In [13]:
df  = pd.concat([data_train,labels_train], axis=1)

#Model AutoML

In [14]:
predictor = TabularPredictor(label='Y', path='ag_experiment',eval_metric = 'balanced_accuracy') # Create an AutoGluon TabularPredictor

In [15]:
predictor.fit(df) # Fit AutoGluon on the training data

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "ag_experiment"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.5
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
Memory Avail:       36.00 GB / 47.85 GB (75.3%)
Disk Space Avail:   816.31 GB / 1667.58 GB (49.0%)
Train Data Rows:    160

	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('int', []) : 500 | ['0', '1', '2', '3', '4', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('int', []) : 500 | ['0', '1', '2', '3', '4', ...]
	1.1s = Fit runtime
	500 features in original data used to generate 500 features in processed data.
	Train Data (Processed) Memory Usage: 6.10 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 1.19s ...
AutoGluon will gauge predictive performance using evaluation metric: 'balanced_accuracy'
	T

In [16]:
df_val  = pd.concat([data_val,labels_val], axis=1)

In [17]:
# Extract evaluation summary
eval_summary = predictor.evaluate(df_val)

In [18]:
print("Balanced Accuracy:", eval_summary['balanced_accuracy'])

Balanced Accuracy: 0.8098822235002876


Dla danych testowych

In [19]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [20]:
pd.set_option('display.max_rows', None)

In [21]:
print(predictor.predict_proba(data_test))

           -1         1
0    0.577617  0.422383
1    0.788296  0.211704
2    0.516956  0.483044
3    0.288343  0.711657
4    0.504205  0.495795
5    0.174880  0.825120
6    0.561680  0.438320
7    0.746769  0.253231
8    0.515456  0.484544
9    0.212039  0.787961
10   0.325612  0.674388
11   0.271525  0.728475
12   0.607745  0.392255
13   0.307606  0.692394
14   0.375267  0.624733
15   0.767191  0.232809
16   0.627638  0.372362
17   0.180392  0.819608
18   0.448164  0.551836
19   0.762269  0.237731
20   0.506211  0.493789
21   0.661234  0.338766
22   0.781464  0.218536
23   0.369003  0.630997
24   0.312259  0.687741
25   0.264752  0.735248
26   0.656871  0.343129
27   0.640535  0.359465
28   0.419197  0.580803
29   0.767449  0.232551
30   0.742147  0.257853
31   0.472546  0.527454
32   0.296247  0.703753
33   0.492297  0.507703
34   0.437463  0.562537
35   0.584876  0.415124
36   0.440762  0.559238
37   0.168899  0.831101
38   0.429286  0.570714
39   0.326326  0.673674
40   0.454640  0

#Model ręczny

In [22]:
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold

##Selekcja zmiennych

###Boruta

In [23]:
#pip install boruta

In [24]:
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

In [25]:
model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

In [26]:
feat_selector = BorutaPy(
    verbose=2,
    estimator=model,
    n_estimators='auto',
    max_iter=10
)

In [27]:
feat_selector.fit(np.array(data_train), np.array(labels_train).ravel())

Iteration: 	1 / 10
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	2 / 10
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	3 / 10
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	4 / 10
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	5 / 10
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	6 / 10
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	7 / 10
Confirmed: 	0
Tentative: 	500
Rejected: 	0
Iteration: 	8 / 10
Confirmed: 	0
Tentative: 	21
Rejected: 	479
Iteration: 	9 / 10
Confirmed: 	17
Tentative: 	4
Rejected: 	479


BorutaPy finished running.

Iteration: 	10 / 10
Confirmed: 	17
Tentative: 	2
Rejected: 	479


BorutaPy(estimator=RandomForestRegressor(max_depth=5, n_estimators=129,
                                         random_state=RandomState(MT19937) at 0x238C43D2E40),
         max_iter=10, n_estimators='auto',
         random_state=RandomState(MT19937) at 0x238C43D2E40, verbose=2)

In [28]:
# print support and ranking for each feature
print("\n------Support and Ranking for each feature------")
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        print("Passes the test: ", data_train.columns[i],
              " - Ranking: ", feat_selector.ranking_[i])
    else:
        print("Doesn't pass the test: ",
              data_train.columns[i], " - Ranking: ", feat_selector.ranking_[i])


------Support and Ranking for each feature------
Doesn't pass the test:  0  - Ranking:  446
Doesn't pass the test:  1  - Ranking:  325
Doesn't pass the test:  2  - Ranking:  284
Doesn't pass the test:  3  - Ranking:  369
Doesn't pass the test:  4  - Ranking:  49
Doesn't pass the test:  5  - Ranking:  297
Doesn't pass the test:  6  - Ranking:  279
Doesn't pass the test:  7  - Ranking:  476
Doesn't pass the test:  8  - Ranking:  388
Doesn't pass the test:  9  - Ranking:  251
Doesn't pass the test:  10  - Ranking:  150
Doesn't pass the test:  11  - Ranking:  207
Doesn't pass the test:  12  - Ranking:  266
Doesn't pass the test:  13  - Ranking:  364
Doesn't pass the test:  14  - Ranking:  185
Doesn't pass the test:  15  - Ranking:  323
Doesn't pass the test:  16  - Ranking:  69
Doesn't pass the test:  17  - Ranking:  4
Doesn't pass the test:  18  - Ranking:  59
Doesn't pass the test:  19  - Ranking:  416
Doesn't pass the test:  20  - Ranking:  159
Doesn't pass the test:  21  - Ranking:  3

In [29]:
df_boruta = feat_selector.transform(np.array(data_train))

In [30]:
df_boruta_csv = pd.DataFrame(df_boruta)

In [33]:
df_boruta_csv.to_csv('./content/sample_data.csv', index=False)

In [34]:
boruta_test = feat_selector.transform(np.array(data_test))

In [35]:
boruta_val = feat_selector.transform(np.array(data_val))

##Fitting the models

In [36]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, balanced_accuracy_score

###GridSearch

In [37]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)

In [38]:
scorer = make_scorer(balanced_accuracy_score)

In [39]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(labels_train)

In [40]:
def score_grid_model(model, grid_param, x_train, y_train, x_test):
  grid_search = GridSearchCV(model, param_grid=grid_param, scoring=scorer,cv=cv, n_jobs=1)
  grid_search.fit(x_train, y_train)
  best_estimator = grid_search.best_estimator_
  scores = cross_val_score(best_estimator, x_train, y_train, cv=cv, scoring=scorer)
  print('Balanced accuracy: %.3f' % mean(scores))
  predictions_proba = pd.DataFrame(grid_search.predict_proba(x_test), columns=grid_search.classes_)
  return predictions_proba, best_estimator


###Random Forest

In [41]:
modelRF = RandomForestClassifier()

In [42]:
param_grid_RF= {'max_depth':[3,10],
              'n_estimators':[10,100,200],
              'max_features':[1,4,7]
           }

In [43]:
RF_result, RF_best = score_grid_model(modelRF,param_grid_RF,data_train,y_train, data_test)

Balanced accuracy: 0.637


In [44]:
RF_result_boruta, RF_best_boruta  = score_grid_model(modelRF,param_grid_RF,df_boruta,y_train, boruta_test)

Balanced accuracy: 0.871


In [45]:
print(RF_result)

            0         1
0    0.500320  0.499680
1    0.557117  0.442883
2    0.532011  0.467989
3    0.446618  0.553382
4    0.492894  0.507106
5    0.418735  0.581265
6    0.537205  0.462795
7    0.545343  0.454657
8    0.444369  0.555631
9    0.389568  0.610432
10   0.473390  0.526610
11   0.426565  0.573435
12   0.452308  0.547692
13   0.507213  0.492787
14   0.522099  0.477901
15   0.498826  0.501174
16   0.557037  0.442963
17   0.388119  0.611881
18   0.558072  0.441928
19   0.472422  0.527578
20   0.480422  0.519578
21   0.594940  0.405060
22   0.586315  0.413685
23   0.551027  0.448973
24   0.534802  0.465198
25   0.512758  0.487242
26   0.538753  0.461247
27   0.476221  0.523779
28   0.519231  0.480769
29   0.609712  0.390288
30   0.571557  0.428443
31   0.548897  0.451103
32   0.489423  0.510577
33   0.469277  0.530723
34   0.479121  0.520879
35   0.485358  0.514642
36   0.562048  0.437952
37   0.396468  0.603532
38   0.514161  0.485839
39   0.492488  0.507512
40   0.478534  0

In [46]:
print(RF_result_boruta)

            0         1
0    0.870566  0.129434
1    0.955274  0.044726
2    0.529267  0.470733
3    0.011824  0.988176
4    0.634487  0.365513
5    0.014449  0.985551
6    0.738321  0.261679
7    0.826926  0.173074
8    0.748623  0.251377
9    0.062370  0.937630
10   0.073098  0.926902
11   0.059386  0.940614
12   0.860931  0.139069
13   0.171559  0.828441
14   0.094330  0.905670
15   0.936400  0.063600
16   0.672766  0.327234
17   0.009239  0.990761
18   0.543792  0.456208
19   0.974714  0.025286
20   0.573214  0.426786
21   0.952313  0.047687
22   0.969152  0.030848
23   0.276419  0.723581
24   0.148257  0.851743
25   0.014661  0.985339
26   0.588502  0.411498
27   0.720100  0.279900
28   0.523232  0.476768
29   0.984749  0.015251
30   0.940014  0.059986
31   0.132697  0.867303
32   0.266268  0.733732
33   0.660658  0.339342
34   0.202469  0.797531
35   0.819379  0.180621
36   0.267892  0.732108
37   0.016541  0.983459
38   0.259313  0.740687
39   0.095669  0.904331
40   0.176733  0

###XGBoost

In [47]:
from xgboost import XGBClassifier

In [48]:
modelXGB = XGBClassifier()

In [49]:
param_grid_XGB = {
    'max_depth': range(2, 10, 3),
    'n_estimators': range(60, 200, 70),
    'learning_rate': [0.1, 0.01, 0.05]
}

In [50]:
XGB_result, XGB_best = score_grid_model(modelXGB,param_grid_XGB,data_train,y_train, data_test)

Balanced accuracy: 0.817


In [51]:
XGB_result_boruta, XGB_best_boruta = score_grid_model(modelXGB,param_grid_XGB,df_boruta,y_train, boruta_test)

Balanced accuracy: 0.866


In [52]:
print(XGB_result)

            0         1
0    0.894289  0.105711
1    0.930283  0.069717
2    0.219352  0.780648
3    0.056310  0.943690
4    0.370661  0.629339
5    0.046628  0.953372
6    0.362310  0.637690
7    0.869963  0.130037
8    0.887797  0.112203
9    0.054592  0.945408
10   0.150760  0.849240
11   0.216687  0.783313
12   0.322552  0.677448
13   0.115523  0.884477
14   0.063531  0.936469
15   0.843420  0.156580
16   0.905691  0.094309
17   0.086225  0.913775
18   0.481514  0.518486
19   0.877140  0.122860
20   0.125695  0.874305
21   0.954864  0.045136
22   0.917311  0.082689
23   0.228180  0.771820
24   0.082660  0.917340
25   0.058784  0.941216
26   0.936696  0.063304
27   0.714105  0.285895
28   0.421463  0.578537
29   0.957799  0.042201
30   0.421204  0.578796
31   0.139453  0.860547
32   0.577337  0.422663
33   0.782143  0.217857
34   0.133379  0.866621
35   0.740110  0.259890
36   0.072256  0.927744
37   0.049786  0.950214
38   0.297620  0.702380
39   0.243831  0.756169
40   0.168646  0

In [53]:
print(XGB_result_boruta)

            0         1
0    0.953454  0.046546
1    0.929746  0.070254
2    0.241864  0.758136
3    0.007550  0.992450
4    0.635281  0.364719
5    0.005934  0.994066
6    0.813215  0.186785
7    0.839730  0.160270
8    0.813936  0.186064
9    0.004635  0.995365
10   0.026555  0.973445
11   0.020025  0.979975
12   0.965526  0.034474
13   0.024599  0.975401
14   0.024404  0.975596
15   0.979625  0.020375
16   0.952778  0.047222
17   0.006848  0.993152
18   0.279701  0.720299
19   0.992846  0.007154
20   0.620654  0.379346
21   0.990230  0.009770
22   0.977188  0.022812
23   0.065827  0.934173
24   0.048879  0.951121
25   0.003138  0.996862
26   0.702402  0.297598
27   0.717544  0.282456
28   0.434227  0.565773
29   0.996692  0.003308
30   0.946719  0.053281
31   0.013805  0.986195
32   0.019140  0.980860
33   0.911710  0.088290
34   0.215368  0.784632
35   0.938964  0.061036
36   0.204961  0.795039
37   0.002566  0.997434
38   0.351767  0.648233
39   0.023196  0.976804
40   0.049723  0

###LightGBM

In [54]:
#%pip install lightgbm

In [55]:
import lightgbm as lgb #scikit-learn implementation
from lightgbm import LGBMClassifier

In [56]:
lightGMB = LGBMClassifier()

In [57]:
param_grid_LGMB = {
    'objective': ['binary'],
    'boosting_type': ['gbdt'],
    'num_leaves': [20, 30],
    'learning_rate':  [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
}

In [58]:
LGBM_result, LGBM_best = score_grid_model(lightGMB,param_grid_LGMB,data_train,y_train, data_test)

[LightGBM] [Info] Number of positive: 722, number of negative: 718
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005767 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 53596
[LightGBM] [Info] Number of data points in the train set: 1440, number of used features: 500
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501389 -> initscore=0.005556
[LightGBM] [Info] Start training from score 0.005556
[LightGBM] [Info] Number of positive: 722, number of negative: 718
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009086 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 53603
[LightGBM] [Info] Number of data points in the train set: 1440, number of used features: 500
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501389 -> initscore=0.005556
[LightGBM] [Info] Start training from score 0.005556
[LightGBM] [Info] Numb

In [59]:
LGBM_result_boruta, LGMB_best_boruta = score_grid_model(lightGMB,param_grid_LGMB,df_boruta,y_train, boruta_test)

[LightGBM] [Info] Number of positive: 722, number of negative: 718
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000210 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3316
[LightGBM] [Info] Number of data points in the train set: 1440, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501389 -> initscore=0.005556
[LightGBM] [Info] Start training from score 0.005556
[LightGBM] [Info] Number of positive: 722, number of negative: 718
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000342 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3307
[LightGBM] [Info] Number of data points in the train set: 1440, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501389 -> initscore=0.005556
[LightGBM] [Info] Start training from score 0.005556
[LightGBM] [Info] Number o

In [60]:
print(LGBM_result)

            0         1
0    0.611017  0.388983
1    0.811578  0.188422
2    0.267334  0.732666
3    0.194043  0.805957
4    0.402055  0.597945
5    0.091939  0.908061
6    0.398381  0.601619
7    0.719838  0.280162
8    0.768879  0.231121
9    0.125553  0.874447
10   0.238450  0.761550
11   0.125522  0.874478
12   0.375546  0.624454
13   0.266043  0.733957
14   0.224549  0.775451
15   0.823101  0.176899
16   0.796642  0.203358
17   0.143577  0.856423
18   0.483143  0.516857
19   0.749964  0.250036
20   0.255902  0.744098
21   0.844378  0.155622
22   0.886092  0.113908
23   0.368236  0.631764
24   0.134568  0.865432
25   0.127716  0.872284
26   0.806352  0.193648
27   0.549517  0.450483
28   0.314325  0.685675
29   0.875767  0.124233
30   0.790854  0.209146
31   0.194637  0.805363
32   0.309870  0.690130
33   0.680482  0.319518
34   0.401014  0.598986
35   0.555703  0.444297
36   0.285655  0.714345
37   0.112933  0.887067
38   0.345596  0.654404
39   0.285787  0.714213
40   0.234130  0

In [61]:
print(LGBM_result_boruta)

            0         1
0    0.992898  0.007102
1    0.977894  0.022106
2    0.502571  0.497429
3    0.001404  0.998596
4    0.571923  0.428077
5    0.000309  0.999691
6    0.969663  0.030337
7    0.936501  0.063499
8    0.575645  0.424355
9    0.003212  0.996788
10   0.002678  0.997322
11   0.011436  0.988564
12   0.980343  0.019657
13   0.011618  0.988382
14   0.015336  0.984664
15   0.998283  0.001717
16   0.963479  0.036521
17   0.000102  0.999898
18   0.125826  0.874174
19   0.999330  0.000670
20   0.667421  0.332579
21   0.999068  0.000932
22   0.997090  0.002910
23   0.027557  0.972443
24   0.002901  0.997099
25   0.000307  0.999693
26   0.466399  0.533601
27   0.902312  0.097688
28   0.311816  0.688184
29   0.999571  0.000429
30   0.983559  0.016441
31   0.000553  0.999447
32   0.003583  0.996417
33   0.914530  0.085470
34   0.335945  0.664055
35   0.980943  0.019057
36   0.272187  0.727813
37   0.000128  0.999872
38   0.188134  0.811866
39   0.005634  0.994366
40   0.010068  0

## Stacking

In [62]:
from sklearn.linear_model import LogisticRegression

In [66]:
# Make predictions on the validation set for boruta chosen features
RF_pred = RF_best_boruta.predict(boruta_val)
XGB_pred = XGB_best_boruta.predict(boruta_val)
LGBM_pred = LGMB_best_boruta.predict(boruta_val)

In [67]:
# Combine the predictions of the base models into a single feature matrix
data_val_meta = np.column_stack((RF_pred, XGB_pred, LGBM_pred))

# Train the meta-model on the combined feature matrix and the target values
meta_model = LogisticRegression()
meta_model.fit(data_val_meta, labels_val)

d:\Data\Dokumenty\kodzenie\Jula\.venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [69]:
# Make predictions on the test set
RF_pred_test = RF_best_boruta.predict(boruta_test)
XGB_pred_test = XGB_best_boruta.predict(boruta_test)
LGBM_pred_test = LGMB_best_boruta.predict(boruta_test)

In [70]:
data_test_meta = np.column_stack((RF_pred_test,XGB_pred_test,LGBM_pred_test))

In [71]:
label_meta_pred = meta_model.predict_proba(data_test_meta)

In [73]:
print(pd.DataFrame(label_meta_pred))

            0         1
0    0.898412  0.101588
1    0.898412  0.101588
2    0.713504  0.286496
3    0.108559  0.891441
4    0.898412  0.101588
5    0.108559  0.891441
6    0.898412  0.101588
7    0.898412  0.101588
8    0.898412  0.101588
9    0.108559  0.891441
10   0.108559  0.891441
11   0.108559  0.891441
12   0.898412  0.101588
13   0.108559  0.891441
14   0.108559  0.891441
15   0.898412  0.101588
16   0.898412  0.101588
17   0.108559  0.891441
18   0.355723  0.644277
19   0.898412  0.101588
20   0.898412  0.101588
21   0.898412  0.101588
22   0.898412  0.101588
23   0.108559  0.891441
24   0.108559  0.891441
25   0.108559  0.891441
26   0.662233  0.337767
27   0.898412  0.101588
28   0.355723  0.644277
29   0.898412  0.101588
30   0.898412  0.101588
31   0.108559  0.891441
32   0.108559  0.891441
33   0.898412  0.101588
34   0.108559  0.891441
35   0.898412  0.101588
36   0.108559  0.891441
37   0.108559  0.891441
38   0.108559  0.891441
39   0.108559  0.891441
40   0.108559  0